In [49]:
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
dev

device(type='cuda')

* TF(term frequency) : 어떤 단어가 특정 문서에서 얼마나 많이 쓰였는지 나타내는 빈도
* DF(document frequency) : 특정 단어가 나타난 문서의 수
* IDF(inverse document frequency) : 전체 문서 수$(N)$를 해당 단어의 DF로 나눈 뒤 로그를 취한 값. 클 수록 특이한 단어라는 의미

$$ \text{TF-IDF}(w) = \text{TF}(w) \times \log\frac{N}{\text{DF}(w)} $$

어떤 단어의 주제 예측 능력이 강할 수록 TF-IDF값이 커지고 그 반대의 경우 작아진다.

In [2]:
from preprocess import get_tokenizer

/home/inhwan/Python/Pytorch_ev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/inhwan/Python/Pytorch_ev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/inhwan/Python/Pytorch_ev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/inhwan/Python/Pytorch_ev/lib/python3.7/site-packages

In [3]:
corpus_fname = 'data/processed/processed_blog.txt'
tokenizer = get_tokenizer('mecab')

In [4]:
titles, raw_corpus, noun_corpus = [], [], []
with open(corpus_fname, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            title, document = line.strip().split('\u241E')
            titles.append(title)
            raw_corpus.append(document)
            nouns = tokenizer.nouns(document)
            noun_corpus.append(' '.join(nouns))
        except:
            continue
        

In [5]:
raw_corpus[0][:300]

' 이번 글에서는 최대엔트로피모델(Maximum Entropy model)의 파라메터 추정을 살펴보도록 하겠습니다. 이 글은 기본적으로 [이곳]()을 참고하였습니다. 그럼 시작하겠습니다.   ## 모델 정의 최대엔트로피 모델은 다음과 같이 정의됩니다.  $$ { P }_{ \\Lambda }(y|x)=\\frac { { exp( }\\sum _{ i }^{ }{ { \\lambda }_{ i }{ f }_{ i }\\left( x,y \\right) } ) }{ \\sum _{ y }^{ }{ { exp( }\\sum _{ i }^{ }{ { \\l'

In [6]:
noun_corpus[0][:100]

'이번 글 최대 엔트로피 모델 파라 메터 추정 글 기본 이곳 참고 시작 모델 정의 최대 엔트로피 모델 다음 정의 위 식 때 값 반환 함수 자질 벡터 번 값 중요 가중치 요소 가중치 '

사이킷런의 TfidfVectorizer를 이용하여 생성한 말뭉치의 명사들에 대하여 TF-IDF 행렬을 생성하자.

TfidfVectorizer의 옵션에 대해서는 다음 글을 참고.<br>
https://datascienceschool.net/view-notebook/3e7aadbf88ed4f0d87a76f9ddc925d69/

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    min_df=1, # document frequency가 1 이상
    ngram_range=(1,1), # 좌우 (1,1)개씩 고려
    lowercase=True,
    tokenizer=lambda x: x.split())
input_matrix = vectorizer.fit_transform(noun_corpus) # 명사들에 대한 TF-IDF 행렬 구하기

In [26]:
input_matrix

<204x37143 sparse matrix of type '<class 'numpy.float64'>'
	with 76870 stored elements in Compressed Sparse Row format>

In [8]:
input_matrix.shape # 행은 문서, 열은 단어에 대응

(204, 37143)

In [9]:
len(vectorizer.vocabulary_)

37143

In [10]:
id2vocab = {vectorizer.vocabulary_[token]:token
           for token in vectorizer.vocabulary_.keys()}
# curr_doc : 말뭉치 첫 번째 문서의 TF-IDF 행렬
curr_doc, result = input_matrix[0], []

In [21]:
curr_doc.indices[:10], curr_doc.data[:10]

(array([30054, 21719, 17148, 26014, 33661, 20188, 19879, 23540, 22470,
        27861], dtype=int32),
 array([0.02321873, 0.06195969, 0.05037386, 0.06943096, 0.08838695,
        0.03324375, 0.03535159, 0.09099347, 0.02587359, 0.03601874]))

In [11]:
# curr_doc에서 TF-IDF값이 0이 아닌 요소들을 내림차순 정렬
for idx, el in zip(curr_doc.indices, curr_doc.data):
    result.append((id2vocab[idx], el))

In [30]:
result[:10]

[('점', 0.02321872527560292),
 ('뺄셈', 0.06195969255574101),
 ('덧셈', 0.05037385883988076),
 ('업데이트', 0.06943095628902349),
 ('터', 0.08838695358356398),
 ('방향', 0.033243751333481585),
 ('반대', 0.03535158640207159),
 ('손실', 0.09099347000775752),
 ('생각', 0.02587358775480135),
 ('유사', 0.036018736356608516)]

In [31]:
sorted(result, key=lambda x:x[1], reverse=True)[:10] # 각 문서를 대표하는 단어들이라고 볼 수 있다

[('우도', 0.30935433754247393),
 ('최대', 0.2644197269001561),
 ('모델', 0.21509543930315736),
 ('디언', 0.20954601175351925),
 ('엔트로피', 0.20954601175351925),
 ('트', 0.2020801317026838),
 ('메터', 0.18998546457990625),
 ('파라', 0.18998546457990625),
 ('확률분포', 0.17931834019736734),
 ('디센트', 0.1740779030970291)]

In [34]:
titles[2]

'word2vec'

In [36]:
input_matrix[0]

<1x37143 sparse matrix of type '<class 'numpy.float64'>'
	with 106 stored elements in Compressed Sparse Row format>

이렇게 생성된 (204, 37413) 크기의 TF-IDF 행렬에 SVD를 수행하여 (204, 100) 크기의 밀집 행렬을 얻자.

In [37]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=100)
vecs = svd.fit_transform(input_matrix)

In [39]:
vecs.shape

(204, 100)

In [46]:
vecs[0]

array([ 4.86747266e-01, -9.56053051e-02, -3.27831811e-01, -9.30603933e-02,
       -1.52408165e-01,  5.70177265e-02,  1.82468966e-01, -6.12313432e-02,
       -1.60263310e-01,  1.18883749e-01,  3.32639479e-02,  1.05234070e-01,
        7.58770191e-02,  4.28242403e-02, -1.16404879e-01,  6.34770148e-02,
       -1.64026660e-02,  3.48644679e-02, -2.30410757e-01, -1.06132091e-01,
        1.57809800e-01, -1.09577114e-01, -3.96189500e-02, -1.51988827e-01,
       -1.04899695e-02, -7.12179941e-02, -5.24978097e-02,  7.59890032e-02,
       -4.45468611e-02,  1.58693679e-01,  9.37446885e-03,  1.55045926e-02,
       -1.51671074e-02, -1.23778800e-01,  2.68083345e-02, -7.25614748e-02,
       -6.73392994e-02, -4.01429908e-03,  1.09203502e-02, -3.03559219e-02,
       -1.55954288e-02, -6.47874112e-03,  2.15010784e-02, -2.59279452e-02,
        6.12138387e-02, -1.36160381e-03, -4.81503075e-03,  1.68810019e-02,
       -6.51446377e-03,  1.71955208e-02, -1.44910331e-02,  1.75601136e-02,
       -6.52873170e-02, -

In [59]:
vecs.shape

(204, 100)

In [60]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
cos(torch.tensor(vecs[0]).view(1,-1), torch.tensor(vecs)).shape

torch.Size([204])

이렇게 생성된 벡터 표현을 저장해두자.

In [53]:
output_fname = 'data/sentence-embeddings/lsa-tfidf/lsa-tfidf.vecs'
with open(output_fname, 'w') as f:
    for doc_idx, vec in enumerate(vecs):
        str_vec = [str(el) for el in vec]
        f.writelines(titles[doc_idx] + "\u241E" + raw_corpus[doc_idx] + '\u241E' + ' '.join(str_vec) + "\n")

이제 문서간의 유사도를 산출해보자.

In [65]:
def similar_docs(vec_mat, titles, index, k=10):
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    
    vec_mat = torch.tensor(vec_mat)
    doc = vec_mat[index].view(1,-1)
    cos_mat = cos(doc, vec_mat)
    sim, indices = torch.topk(cos_mat,k+1)
    
    id_titles = []
    for i in indices:
        if i != index:
            id_titles.append(titles[i])
    return pd.Series(id_titles, np.array(sim[1:].detach()))

In [66]:
similar_docs(vecs, titles, 0)

0.751197        loss
0.731756         MLE
0.686223         CRF
0.620925     unsugen
0.591298    logistic
0.565547    gradient
0.504974         VAE
0.496815     softmax
0.483263    NNtricks
0.479947        MEMs
dtype: object

이제 책에 쓰인 방법대로 유사문서 검색 및 시각화를 해보자.

먼저 다음과 같이 모델을 불러오려고 했으나, tf.contrib의 nccl을 찾을 수 없다고 오류가 떠서 `model/sent_eval.py` 의 `from tune_utils import make_elmo_graph, make_bert_graph` 부분을 주석 처리함.

In [72]:
from models.sent_eval import LSAEvaluator
model = LSAEvaluator('data/sentence-embeddings/lsa-tfidf/lsa-tfidf.vecs')
model.most_similar(doc_id=0)

['maxparam',
 [('loss', 0.7511969326756819),
  ('MLE', 0.7317555129892188),
  ('CRF', 0.6862234059889061),
  ('unsugen', 0.6209250503586896),
  ('logistic', 0.5912982144408041),
  ('gradient', 0.5655466890513458),
  ('VAE', 0.5049736723248607),
  ('softmax', 0.4968154562639956),
  ('NNtricks', 0.48326250454696795),
  ('MEMs', 0.47994716262040377)]]

In [74]:
model.visualize('between')

save @ between-sentences.png


In [75]:
model.visualize('tsne')

save @ sentences.png


<img src='between-sentences.png'>

<img src='sentences.png'>